# Start a chat with LLM

The `Session` object is at the core of lionagi.

A session is an interface to
- manages and logs various messages (system, user, assistant) in a conversation, 
- get AI model inferencing with `Services`
- enable effortless multi-round exchange between many participants. 

A Session object contains one or more `branch`, by default, every session has a `'main'` branch， `session.branches['main']`

- `branch.messages` is a `pd.DataFrame`

lionagi has some default config, you can check them under lionagi.integrations.config

In [ ]:
from IPython.display import Markdown  # for better print

In [2]:
from lionagi.integrations.config import oai_schema

oai_schema["chat/completions"]["config"]

{'model': 'gpt-4-1106-preview',
 'frequency_penalty': 0,
 'max_tokens': None,
 'n': 1,
 'presence_penalty': 0,
 'response_format': {'type': 'text'},
 'seed': None,
 'stop': None,
 'stream': False,
 'temperature': 0.7,
 'top_p': 1,
 'tools': None,
 'tool_choice': 'none',
 'user': None}

In [ ]:
# create some messages
sys_comedian = "As a comedian, you are sarcastically funny"
instruct1 = (
    "very short joke: a blue whale and a big shark meet at the bar and start dancing"
)

In [3]:
from lionagi import Session

# create a llm conversation with OpenAI gpt-4 (default)
comedian1 = Session(system=sys_comedian)
joke1 = await comedian1.chat(instruction=instruct1, max_token=50)

In [4]:
Markdown(joke1)

You can change a LLM provider by choosing a pre-configured service, or create your own

In [6]:
from lionagi import Services

service = Services.OpenRouter(
    max_tokens=1000, max_requests=10, interval=60
)  # set tokens rate limit
model = "mistralai/mixtral-8x7b-instruct"  # use any model from openrouter, you need an openrouter api key

comedian2 = Session(system=sys_comedian, service=service)
joke2 = await comedian2.chat(instruction=instruct1, model=model, max_token=50)

In [8]:
Markdown(joke2)


{"joke": "The blue whale and the big shark had a whale of a time at the bar!"}

In [9]:
# The following three implementation are equivalent
# for getting the messages DataFrame from the default branch

"""
comedian2.messages
comedian2.default_branch.messages
comedian2.branches['main'].messages
"""

comedian2.branches["main"].messages

,node_id,role,name,timestamp,content
0,ac6cfc4df0914941161171b333771c3d,system,system,2024-01-18 13:15:20.380715,"{""system_info"": ""As a comedian, you are sarcas..."
1,f536c4c76c862dee8ec133756e67098f,user,user,2024-01-18 13:15:20.381720,"{""instruction"": ""very short joke: a blue whale..."
2,b38068a7e532dd71a74e2441791a817f,assistant,assistant,2024-01-18 13:15:21.981097,"{""response"": ""\n{\""joke\"": \""The blue whale an..."


In [ ]:
# check the default branch information
comedian2.default_branch.info